### Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
import warnings
warnings.filterwarnings('ignore')

### Initialize the dataframe

In [41]:
df = pd.read_excel("./kantipurEngi.xlsx", index_col=0)
df.head()

,Godavari,Lele,Khumaltar,Sankhu,Panipokhari (Kathmandu),Nagarkot,Bhaktapur,Changu Narayan,Chapa Gaun,Buddhanilakantha,Khokana,Sundarijal (Mulkharka),Naikap,Jitpurphedhi,Nangkhel,Kathmandu Airport
Time,,,,,,,,,,,,,,,,
01/01/1994 03:00:00,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
01/02/1994 03:00:00,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
01/03/1994 03:00:00,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
01/04/1994 03:00:00,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
01/05/1994 03:00:00,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0


### Inspecting the dataframe

In [42]:
df.columns

Index(['Godavari', 'Lele', 'Khumaltar', 'Sankhu', 'Panipokhari (Kathmandu)',
       'Nagarkot', 'Bhaktapur', 'Changu Narayan', 'Chapa Gaun',
       'Buddhanilakantha', 'Khokana', 'Sundarijal (Mulkharka)', 'Naikap',
       'Jitpurphedhi', 'Nangkhel', 'Kathmandu Airport'],
      dtype='object')

In [43]:
df.isnull().sum()

Godavari                    311
Lele                        324
Khumaltar                    75
Sankhu                      432
Panipokhari (Kathmandu)     611
Nagarkot                     75
Bhaktapur                   359
Changu Narayan               97
Chapa Gaun                  216
Buddhanilakantha           2814
Khokana                       6
Sundarijal (Mulkharka)      144
Naikap                     1366
Jitpurphedhi               2521
Nangkhel                   3122
Kathmandu Airport            62
dtype: int64

**We can observe that stations installed at Naikap, Jitpurphedhi and Nangkhel are missing data in high numbers.**

In [44]:
df.dtypes

Godavari                   float64
Lele                       float64
Khumaltar                  float64
Sankhu                     float64
Panipokhari (Kathmandu)    float64
Nagarkot                   float64
Bhaktapur                  float64
Changu Narayan             float64
Chapa Gaun                 float64
Buddhanilakantha           float64
Khokana                    float64
Sundarijal (Mulkharka)     float64
Naikap                     float64
Jitpurphedhi               float64
Nangkhel                   float64
Kathmandu Airport          float64
dtype: object

In [45]:
df.index = pd.to_datetime(df.index)
df.head()

,Godavari,Lele,Khumaltar,Sankhu,Panipokhari (Kathmandu),Nagarkot,Bhaktapur,Changu Narayan,Chapa Gaun,Buddhanilakantha,Khokana,Sundarijal (Mulkharka),Naikap,Jitpurphedhi,Nangkhel,Kathmandu Airport
Time,,,,,,,,,,,,,,,,
1994-01-01 03:00:00,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
1994-01-02 03:00:00,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
1994-01-03 03:00:00,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
1994-01-04 03:00:00,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
1994-01-05 03:00:00,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0


In [46]:
df.shape

(11202, 16)

In [47]:
print("The dataset contains data of", 11202/365, "years.")

The dataset contains data of 30.69041095890411 years.


### Kriging Interpolation To Impute Missing Data

In [48]:
# Define coordinates of stations (latitude, longitude)
STATION_COORDS  = {
    "Godavari": (27.5744859, 85.4003349),
    "Lele": (27.5541, 85.3632),
    "Khumaltar": (27.6461, 85.3262),
    "Sankhu": (27.7492, 85.4468),
    "Panipokhari (Kathmandu)": (27.7194, 85.3230),
    "Nagarkot": (27.7159, 85.5206),
    "Bhaktapur": (27.6725, 85.4298),
    "Changu Narayan": (27.7167, 85.4322),
    "Chapa Gaun": (27.6186, 85.3669),
    "Buddhanilakantha": (27.7798, 85.3677),
    "Khokana": (27.6242, 85.3098),
    "Sundarijal (Mulkharka)": (27.7776, 85.4073),
    "Naikap": (27.6875, 85.2574),
    "Jitpurphedhi": (27.7743, 85.3718),
    "Nangkhel": (27.6678, 85.4312),
    "Kathmandu Airport": (27.6987, 85.3592),
}

In [49]:
def interpolate_station_data(df, min_stations=3):
    """
    Interpolate missing values using Gaussian Process Regression (Kriging).
    """
    print("Starting interpolation process...")
    result_df = df.copy()
    
    # Ensure coordinates are precomputed once
    stations = list(STATION_COORDS.keys())
    all_coords = np.array([STATION_COORDS[station] for station in stations])
    
    # Scale coordinates to kilometers for better numerical stability
    ref_lat, ref_lon = all_coords.mean(axis=0)
    coords_km = np.zeros_like(all_coords)
    R = 6371  # Earth radius in km
    
    # Convert to kilometers from reference point
    coords_km[:, 0] = (all_coords[:, 0] - ref_lat) * 111  # 1 degree lat ≈ 111 km
    coords_km[:, 1] = (all_coords[:, 1] - ref_lon) * 111 * np.cos(np.radians(ref_lat))
    
    # Initialize Gaussian Process model
    kernel = RBF(length_scale=10.0) + WhiteKernel(noise_level=1.0)
    gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=2)
    
    # Process each row with missing values
    missing_dates_mask = df.isna().any(axis=1)
    dates_to_process = df.index[missing_dates_mask].tolist()
    
    print(f"Total dates to process: {len(dates_to_process)}")
    
    for date in tqdm(dates_to_process, desc="Processing Dates"):
        try:
            row = df.loc[date]
            
            # Ensure row is a 1D Series
            if isinstance(row, pd.DataFrame):
                row = row.squeeze()
            
            # Skip if we don't have enough non-missing values
            valid_mask = row.notna()
            if valid_mask.sum() < min_stations:
                continue
            
            # Get known values and their coordinates
            valid_stations = row.index[valid_mask].tolist()
            valid_values = row[valid_mask].values
            valid_coords = np.array([coords_km[stations.index(station)] for station in valid_stations])
            
            # Get missing stations
            missing_mask = row.isna()
            missing_stations = row.index[missing_mask].tolist()
            if not missing_stations:
                continue
            
            missing_coords = np.array([coords_km[stations.index(station)] for station in missing_stations])
            
            # Fit the Gaussian Process model
            gpr.fit(valid_coords, valid_values)
            
            # Predict missing values
            predicted_values = gpr.predict(missing_coords)
            
            # Update the values
            result_df.loc[date, missing_stations] = predicted_values
            
        except Exception as e:
            print(f"GPR failed for date {date}: {str(e)}")
            # Fallback to inverse distance weighting
            for j, station in enumerate(missing_stations):
                dists = np.sqrt(np.sum((valid_coords - missing_coords[j])**2, axis=1))
                weights = 1 / (dists + 1e-6)**2
                weights /= weights.sum()
                result_df.loc[date, station] = np.sum(valid_values * weights)
    
    print("Performing temporal interpolation...")
    
    # Fill remaining gaps with temporal methods
    result_df = result_df.fillna(method='ffill', limit=7)
    result_df = result_df.fillna(method='bfill', limit=7)
    result_df = result_df.interpolate(method='time', limit_direction='both')
    
    print("Interpolation complete!")
    return result_df


In [50]:
def validate_interpolation(original_df, interpolated_df):
    """
    Generate validation statistics.
    """
    missing_original = original_df.isna().sum()
    missing_interpolated = interpolated_df.isna().sum()
    percent_filled = ((missing_original - missing_interpolated) / missing_original * 100).round(2)
    
    stats = pd.DataFrame({
        'Original Missing': missing_original,
        'Remaining Missing': missing_interpolated,
        'Percent Filled': percent_filled
    })
    
    return stats

In [51]:
# Perform interpolation
interpolated_df = interpolate_station_data(df)

Starting interpolation process...
Total dates to process: 6241


Processing Dates: 100%|████████████████████| 6241/6241 [02:10<00:00, 47.84it/s]

Performing temporal interpolation...
Interpolation complete!


In [52]:
stats = validate_interpolation(df, interpolated_df)
print("\nInterpolation Statistics:")
print(stats)


Interpolation Statistics:
                         Original Missing  Remaining Missing  Percent Filled
Godavari                              311                  0           100.0
Lele                                  324                  0           100.0
Khumaltar                              75                  0           100.0
Sankhu                                432                  0           100.0
Panipokhari (Kathmandu)               611                  0           100.0
Nagarkot                               75                  0           100.0
Bhaktapur                             359                  0           100.0
Changu Narayan                         97                  0           100.0
Chapa Gaun                            216                  0           100.0
Buddhanilakantha                     2814                  0           100.0
Khokana                                 6                  0           100.0
Sundarijal (Mulkharka)                144        

In [ ]:
interpolated_df.to_csv("interpolated_data.csv")

In [82]:
interpolated_df = pd.read_csv("interpolated_data.csv", index_col=0)

In [83]:
interpolated_df.head()

,Godavari,Lele,Khumaltar,Sankhu,Panipokhari (Kathmandu),Nagarkot,Bhaktapur,Changu Narayan,Chapa Gaun,Buddhanilakantha,Khokana,Sundarijal (Mulkharka),Naikap,Jitpurphedhi,Nangkhel,Kathmandu Airport
Time,,,,,,,,,,,,,,,,
1994-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1994-01-02 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1994-01-03 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1994-01-04 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1994-01-05 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Further processing the Dataframe

In [84]:
# Round to 2 decimal places
processed_df = interpolated_df.round(2)

# Replace negative values with 0
processed_df = processed_df.clip(lower=0)

In [85]:
interpolated_df

,Godavari,Lele,Khumaltar,Sankhu,Panipokhari (Kathmandu),Nagarkot,Bhaktapur,Changu Narayan,Chapa Gaun,Buddhanilakantha,Khokana,Sundarijal (Mulkharka),Naikap,Jitpurphedhi,Nangkhel,Kathmandu Airport
Time,,,,,,,,,,,,,,,,
1994-01-01 03:00:00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1994-01-02 03:00:00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1994-01-03 03:00:00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1994-01-04 03:00:00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1994-01-05 03:00:00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-27 03:00:00,4.500000,1.695142,3.4,0.975665,1.046842,1.500000,3.7,0.01,1.676130,0.300000,2.200000,0.776033,0.010000,0.765318,1.506837,0.2
2024-06-28 03:00:00,9.300000,0.000000,10.6,0.000000,0.000000,0.000000,14.1,15.20,0.000000,88.400000,0.000000,0.000000,0.000000,0.000000,0.000000,19.5
2024-06-29 03:00:00,0.000000,0.391429,6.3,0.391449,0.391450,0.391438,17.0,10.60,0.391450,0.391444,0.391442,0.391445,0.391434,0.391445,0.391456,12.6


In [86]:
processed_df.to_excel('processed_df.xlsx')